In [1]:
import urllib2
import numpy as np
import json
import pandas as pd
pd.set_option("display.max_columns",999)
from pandas.io.json import json_normalize
import os
import pickle

In [2]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()

In [3]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

In [82]:
#number per page = 30 
start_pg = 3333
end_pg = 10000
total_records = 100000
per_page = 1
n_pages=int(total_records/per_page)

In [83]:
#%run 'Download_iNat_data.ipynb'

In [84]:
#sparkdf = sqlsc.createDataFrame(iNat_df_main)
data = sqlsc.read.json('data/*')

In [85]:
data.count() 

6197

In [86]:
from pyspark.sql import functions as F
#joindf["show"].cast(DoubleType())
t = data.select(data["results.captive"].getItem(0).alias('captive'), 
                data["results.cached_votes_total"].getItem(0).alias('cached_votes_total'),
                data['results.comments_count'].getItem(0).alias('comments_count'),
                data['results.community_taxon_id'].getItem(0).alias('community_taxon_id'),
                data['results.observed_on_details.day'].getItem(0).alias('observed_on_day'),
                data['results.observed_on_details.hour'].getItem(0).alias('observed_on_hour'),
                data['results.observed_on_details.month'].getItem(0).alias('observed_on_month'),
                data['results.observed_on_details.week'].getItem(0).alias('observed_on_week'),
                data['results.observed_on_details.year'].getItem(0).alias('observed_on_year'),
                data['results.faves_count'].getItem(0).alias('faves_count'),
                data['results.geojson.coordinates'].getItem(0).alias('coordinates'),
                data['results.geojson.type'].getItem(0).alias('type'),
                data['results.id'].getItem(0).alias('id'),
                data['results.id_please'].getItem(0).alias('id_please'),
                data['results.identifications_count'].getItem(0).alias('identifications_count'),
                data['results.identifications_most_agree'].getItem(0).alias('identifications_most_agree'),
                data['results.identifications_most_disagree'].getItem(0).alias('identifications_most_disagree'),
                data['results.identifications_some_agree'].getItem(0).alias('identifications_some_agree'),
                data['results.mappable'].getItem(0).alias('mappable'),
                data['results.owners_identification_from_vision'].getItem(0).alias('owners_identification_from_vision'),
                data['results.place_ids'].getItem(0).alias('place_ids'),
                data['results.quality_grade'].getItem(0).alias('quality_grade'),
                'results.species_guess',
                data['results.taxon.ancestor_ids'],#.getItem(0).alias('ancestor_ids'),
                data['results.taxon.complete_rank'],#.getItem(0).alias('complete_rank'),
                'results.taxon.introduced',
                data['results.taxon.extinct'].getItem(0).alias('extinct'),
                data['results.taxon.iconic_taxon_name'],#.getItem(0).alias('iconic_taxon_name'),
                data['results.taxon.min_species_ancestry'],#.getItem(0).alias('min_species_ancestry'),
                data['results.taxon.observations_count'].getItem(0).alias('observations_count'),
                data['results.taxon.rank'],#.getItem(0).alias('rank'),
                data['results.taxon.extinct'].getItem(0).alias('extinct'),
                data['results.taxon.threatened'].getItem(0).alias('threatened'),
                data['results.user.activity_count'].getItem(0).alias('activity_count'),
                data['results.user.identifications_count'].getItem(0).alias('identifications_count')
                   )


In [87]:
t.printSchema()

root
 |-- captive: boolean (nullable = true)
 |-- cached_votes_total: long (nullable = true)
 |-- comments_count: long (nullable = true)
 |-- community_taxon_id: long (nullable = true)
 |-- observed_on_day: long (nullable = true)
 |-- observed_on_hour: long (nullable = true)
 |-- observed_on_month: long (nullable = true)
 |-- observed_on_week: long (nullable = true)
 |-- observed_on_year: long (nullable = true)
 |-- faves_count: long (nullable = true)
 |-- coordinates: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- type: string (nullable = true)
 |-- id: long (nullable = true)
 |-- id_please: boolean (nullable = true)
 |-- identifications_count: long (nullable = true)
 |-- identifications_most_agree: boolean (nullable = true)
 |-- identifications_most_disagree: boolean (nullable = true)
 |-- identifications_some_agree: boolean (nullable = true)
 |-- mappable: boolean (nullable = true)
 |-- owners_identification_from_vision: boolean (nullable = true)
 |-- p

In [88]:
t.select('species_guess').distinct().count()

1884

In [89]:
t.select('species_guess').distinct().show(5)

+--------------------+
|       species_guess|
+--------------------+
| [Tennessee Warbler]|
|  [Great Horned Owl]|
|[Shamrock Orbweaver]|
|[Yellow Sand Verb...|
|     [Poecilochirus]|
+--------------------+
only showing top 5 rows



In [90]:
t.filter(t.identifications_most_agree == True).count(),t.count()

(4038, 6197)

In [91]:
t.groupby('observed_on_year').count().show()

+----------------+-----+
|observed_on_year|count|
+----------------+-----+
|               0|   25|
|            2000|    1|
|            2002|    2|
|            2003|    3|
|            2004|    1|
|            2005|    4|
|            2006|    4|
|            2007|    3|
|            2008|    9|
|            2009|   14|
|            2010|   33|
|            2011|  114|
|            2012|   26|
|            2013|    8|
|            2014|   29|
|            2015|  303|
|            2016|   97|
|            2017|  914|
|            2018| 4607|
+----------------+-----+



In [30]:
taxa = t.groupby('iconic_taxon_name').count().toPandas() 

In [35]:
import plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='tjogesh', api_key='jQCndI6d5M1oYF4mXX7i')

In [54]:
labels = list(taxa.iconic_taxon_name.map(lambda x: x[0]))
values = list(taxa['count'])

fig = {
    'data': [{'labels': labels,
              'values': values,
              'type': 'pie'}],
    'layout': {'title': 'Major organismal groups observed on iNaturalist'}
     }

py.plotly.iplot(fig, filename='taxa_on_iNat')


In [92]:
species= t.groupby('species_guess').count().toPandas() 

In [93]:
species['name'] = species.species_guess.map(lambda x: x[0])
species.dropna(inplace=True)

In [94]:
species.sort_values('count', ascending=False).head(10)

,species_guess,count,name
227,[gilled mushrooms],37,gilled mushrooms
490,[American Coot],36,American Coot
1449,[creosote bush],31,creosote bush
837,[Red-tailed Hawk],27,Red-tailed Hawk
1144,[Mallard],27,Mallard
1202,[White-crowned Sparrow],25,White-crowned Sparrow
1033,[Golden-crowned Sparrow],24,Golden-crowned Sparrow
1720,[Black Phoebe],23,Black Phoebe
9,[Great Egret],23,Great Egret
390,[Great Blue Heron],23,Great Blue Heron


In [95]:
taxa_id = t.crosstab('iconic_taxon_name', 'identifications_most_agree').show() 

+--------------------------------------------+----+-----+
|iconic_taxon_name_identifications_most_agree|true|false|
+--------------------------------------------+----+-----+
|                        WrappedArray(Arac...|  55|   46|
|                        WrappedArray(Inse...| 195|  173|
|                        WrappedArray(Chro...|  19|   55|
|                        WrappedArray(Rept...| 100|   25|
|                        WrappedArray(Prot...|   1|    7|
|                        WrappedArray(Mamm...| 176|   59|
|                          WrappedArray(Aves)|1488|  368|
|                         WrappedArray(Fungi)| 624|  391|
|                        WrappedArray(Anim...|  48|   69|
|                        WrappedArray(Moll...| 111|   67|
|                          WrappedArray(null)|   1|  153|
|                        WrappedArray(Amph...| 143|   34|
|                        WrappedArray(Acti...|  15|    8|
|                        WrappedArray(Plan...|1062|  704|
+-------------

In [97]:
taxa_id['name'] = taxa_id.iconic_taxon_name_identifications_most_agree.map(lambda x: x[0])

AttributeError: 'NoneType' object has no attribute 'iconic_taxon_name_identifications_most_agree'

In [54]:
labels = list(taxa.iconic_taxon_name.map(lambda x: x[0]))
values = list(taxa['count'])

fig = {
    'data': [{'labels': labels,
              'values': values,
              'type': 'pie'}],
    'layout': {'title': 'Major organismal groups observed on iNaturalist'}
     }

py.plotly.iplot(fig, filename='taxa_on_iNat')


In [78]:
t.crosstab('captive', 'identifications_most_agree').show() 

+----------------------------------+----+-----+
|captive_identifications_most_agree|true|false|
+----------------------------------+----+-----+
|                             false| 915|  603|
+----------------------------------+----+-----+



In [79]:
t.crosstab('cached_votes_total', 'identifications_most_agree').show() 

+---------------------------------------------+----+-----+
|cached_votes_total_identifications_most_agree|true|false|
+---------------------------------------------+----+-----+
|                                            1|   4|    1|
|                                            0| 911|  602|
+---------------------------------------------+----+-----+



In [81]:
t.crosstab('observed_on_hour', 'identifications_most_agree').show() 

+-------------------------------------------+----+-----+
|observed_on_hour_identifications_most_agree|true|false|
+-------------------------------------------+----+-----+
|                                          0|  35|   46|
|                                          5|   3|    1|
|                                         10|  80|   56|
|                                         14|  99|   87|
|                                         20|   9|    4|
|                                          1|   2|    0|
|                                          6|   2|    3|
|                                         21|  14|    5|
|                                          9|  70|   40|
|                                         13|  67|   76|
|                                          2|   7|   77|
|                                         17|  27|   12|
|                                         22|  10|    4|
|                                         12| 112|   30|
|                              

In [19]:
#df = t.toPandas() 

In [ ]:
df.to_csv("test.csv")